In [92]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn import linear_model
dtype_dict = {'name':str,'review':str,'rating':int}

In [8]:
products = pd.read_csv('../data/amazon_baby.csv',dtype=dtype_dict)
train_series = pd.read_json('../data/module-2-assignment-train-idx.json')
test_series = pd.read_json('../data/module-2-assignment-test-idx.json')
train_series = list(train_series[0])
test_series = list(test_series[0])

# A function to remove the punctuations from the reviews to make 'good' and 'good!' the same

In [9]:
def remove_punctuations(text):
    import string
    text = text.translate(None,string.punctuation)
    return text

In [10]:
products['review_clean'] = remove_punctuations(products['review'].str)

# Setting Null review with empty review and selecting reviews which are not 3

In [11]:
products = products.fillna({'review':''}) 
products = products[products['rating']!=3]


## Assinging class to the rating column(+1 or -1)

In [12]:
products['sentiment'] = products['rating'].apply(lambda x : 1 if x >3 else -1 )

In [14]:
data_train = products.ix[train_series]
data_test = products.ix[test_series]
data_train = data_train[data_train['name'].isnull() == False]
data_test = data_test[data_test['name'].isnull() == False]
data_test = data_test.fillna({'review':''})


# vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')

# train_matrix = vectorizer.fit_transform(data_train['review_clean'].values.astype('U'))
# test_matrix = vectorizer.transform(data_test['review_clean'].values.astype('U'))


In [18]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']
vectorizer = CountVectorizer(vocabulary=significant_words)
train_matrix = vectorizer.fit_transform(data_train['review_clean'].values.astype('U'))


121011
30209
CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None,
        vocabulary=['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 'work', 'product', 'money', 'would', 'return'])


In [20]:
test_matrix = vectorizer.transform(data_test['review_clean'].values.astype('U'))

In [21]:
simple_model = linear_model.LogisticRegression()
simple_model.fit(train_matrix,data_train['sentiment'])


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [36]:
coeff = simple_model.coef_
W = pd.DataFrame(data=coeff,columns=significant_words)
print W



       love     great      easy       old   little   perfect     loves  \
0  1.362158  0.926066  1.184082  0.123218  0.49935  1.515748  1.744287   

       well      able       car     broke      less      even    waste  \
0  0.470564  0.182177  0.091737 -1.667943 -0.188437 -0.520674 -2.01268   

   disappointed      work   product     money    would    return  
0     -2.354803 -0.637185 -0.310005 -0.884779 -0.33753 -2.020611  


# 10 positive, 10 negative 

In [37]:
data_train['prediction'] = simple_model.predict(train_matrix)
data_test['prediction'] = simple_model.predict(test_matrix)

In [48]:
correct_prediction_train = data_train[data_train['sentiment'] == data_train['prediction']]
correct_prediction_test = data_test[data_test['sentiment'] == data_test['prediction']]


In [50]:
train_acc_simple = float(len(correct_prediction_train))/len(data_train)
test_acc_simple = float(len(correct_prediction_test))/len(data_test)
print "Train accuracy simple model ",train_acc_simple
print "Test accuract simple model", test_acc_simple

Train accuracy simple model  0.864995744189
Test accuract simple model 0.868681518753


In [55]:
majority_class = max(len(data_test[data_test['sentiment'] == 1]),len(data_test[data_test['sentiment'] == 1]))
majority_class_acc = float(majority_class)/len(data_test)
print "Majority class accuracy", majority_class_acc

Majority class accuracy 0.841537290212


In [56]:
vectorizer_sentiments = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix_sentiments = vectorizer_sentiments.fit_transform(data_train['review_clean'].values.astype('U'))

In [58]:
test_matrix_sentiments = vectorizer_sentiments.transform(data_test['review_clean'].values.astype('U'))

In [60]:
sentiment_model= linear_model.LogisticRegression()
sentiment_model.fit(train_matrix_sentiments,data_train['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [64]:
sentiments_coeff = list(sentiment_model.coef_[0,:])
print len(sentiments_coeff)
count = 1
for x in sentiments_coeff:
    if x >= 0 :
        count += 1
print count

113129
79501


# Non zero coeff 79501

In [72]:
sample_test_data = data_test[10:13]
sample_test_data['review'][0:1]

64    Really happy with this purchase. I was looking...
Name: review, dtype: object

In [75]:
sample_test_matrix = vectorizer_sentiments.transform(sample_test_data['review'])
scores = sentiment_model.decision_function(sample_test_matrix)
print scores

[  9.05478641   8.13453431  10.07155764]


In [79]:
sample_prediction = [np.sign(x) for x in scores] 
print sample_prediction

[1.0, 1.0, 1.0]


In [93]:
def probability(score):
    return 1/(1+np.exp(-score))

In [94]:
print [probability(x) for x in scores]

[0.99988318315027835, 0.99970685039990215, 0.9999577370569166]


In [95]:
print sentiment_model.predict_proba(sample_test_matrix)

[[  1.16816850e-04   9.99883183e-01]
 [  2.93149600e-04   9.99706850e-01]
 [  4.22629431e-05   9.99957737e-01]]


In [96]:
data_train['sentiment_prediction'] = sentiment_model.predict(train_matrix_sentiments)
data_test['sentiments_prediction'] = sentiment_model.predict(test_matrix_sentiments)

In [104]:
data_test['sentiments_probability'] = sentiment_model.predict_proba(test_matrix_sentiments)[:,1]
print data_test
data_test = data_test.sort(columns=['sentiments_probability'])
data_test[-20:]

                                                     name  \
131869                     Aqueduck Faucet Extender, Pink   
77728                     Skip Hop Bath Spout Cover, Moby   
132697  Infantino Baby Girl Animals Twist and Fold Act...   
32044   Ginsey Angel Baby Blanket With Halo &amp; Wing...   
160557     Nuby 16 Piece Bath Tub Foam, Animal Characters   
149463  Munchkin Click Lock 2 Count Flip Straw Cup, 9 ...   
21864   North States Supergate Classic Plastic Gate Mo...   
63257   Philips AVENT Basic Baby Monitor with DECT Tec...   
118517      Baby Jogger City Select Single Stroller, Onyx   
135793  Waterproof Bamboo Nursing Pads - Green Flowers...   
46143   Kids Preferred Amazing Baby Feel and Learn - S...   
58025   Avent SCF274 30 BPA Free Electric Steam Steril...   
147719           Evenflo Modern 200 High Chair, Tangerine   
67823   Fisher-Price 2-in-1 Projection Mobile, Preciou...   
106089      Luvable Friends 6 Pack Baby Burp Cloths, Blue   
151592     Nurtria Basic

/home/om/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


,name,review,rating,review_clean,sentiment,prediction,sentiments_prediction,sentiments_probability
123940,Lifenest Sleep System,I know 2 babies who improved their flat heads ...,5.0,I know 2 babies who improved their flat heads ...,1.0,1.0,1.0,1.0
31459,Cosco High Back Booster Car Seat,"I am by no means a brand snob, so when I wante...",4.0,I am by no means a brand snob so when I wanted...,1.0,1.0,1.0,1.0
30724,Baby Buddy: Baby's 1st Toothbrush,This is a perfect 1st toothbrush. My LO loves ...,5.0,This is a perfect 1st toothbrush My LO loves t...,1.0,1.0,1.0,1.0
37377,Inglesina Zippy Stroller Sabbia,I'm always looking for excuses to use mine! Do...,5.0,Im always looking for excuses to use mine Down...,1.0,1.0,1.0,1.0
130374,Evenflo Bounce and Learn Bee Exersaucer,Such a cute excersaucer. I looked around for a...,5.0,Such a cute excersaucer I looked around for aw...,1.0,1.0,1.0,1.0
33110,Lamaze Play &amp; Grow Mortimer the Moose Take...,It's a great teether and is cuddly and holds m...,5.0,Its a great teether and is cuddly and holds my...,1.0,1.0,1.0,1.0
79822,"Simple Wishes Hands-Free Breastpump Bra, Pink,...",This is sooo wonderful. I can now do something...,5.0,This is sooo wonderful I can now do something ...,1.0,1.0,1.0,1.0
60407,"Balboa Baby Dr. Sears Adjustable Sling, Blue P...",Have not used it yet bc I have not given birth...,5.0,Have not used it yet bc I have not given birth...,1.0,1.0,1.0,1.0
68730,"UPPAbaby G-Luxe Stroller, Black Jake",Ok Moms- The lite weight stroller we bought is...,5.0,Ok Moms The lite weight stroller we bought is ...,1.0,1.0,1.0,1.0
135008,Infant Optics DXR-5 2.4 GHz Digital Video Baby...,We bought this baby monitor a little under 1 y...,5.0,We bought this baby monitor a little under 1 y...,1.0,1.0,1.0,1.0


In [105]:
data_test[:20]

,name,review,rating,review_clean,sentiment,prediction,sentiments_prediction,sentiments_probability
164336,"Modern Littles Bold Folding Laundry Basket, So...",This awesome & durable Laundry bin is perfect ...,5.0,This awesome durable Laundry bin is perfect f...,1.0,1.0,1.0,4.324334e-21
36012,"Manhattan Toy Snuggle Pod, Peanut","This item is super cute, soft, and seems well ...",4.0,This item is super cute soft and seems well ma...,1.0,1.0,1.0,9.922759e-18
127392,North American Bear 12&quot; Velveteenie Circu...,"Excellent gift for a newborn. Super soft, no l...",5.0,Excellent gift for a newborn Super soft no loo...,1.0,1.0,1.0,7.453433e-17
16416,BABYBJ&Ouml;RN Active Carrier - Black,We used this carrier until our baby was 8.5 mo...,5.0,We used this carrier until our baby was 85 mon...,1.0,1.0,1.0,1.910418e-15
115710,Safety 1st Alpha Omega Elite Convertible Car S...,"This car seat is so safe and so comfortable, a...",5.0,This car seat is so safe and so comfortable an...,1.0,1.0,1.0,1.098422e-13
93372,"Lamaze Early Development Toy, Captain Calamari",This is the perfect toy design! The legs are s...,5.0,This is the perfect toy design The legs are si...,1.0,1.0,1.0,1.735050e-12
81243,SheetWorld - Baby Pillow Case - Percale Pillow...,I'm pretty sure this pillow case says it is 10...,4.0,Im pretty sure this pillow case says it is 100...,1.0,1.0,-1.0,3.242297e-12
75327,"Munchkin Jelly Bean Reversible Sling, Champagn...",I found this sling to be much better than the ...,4.0,I found this sling to be much better than the ...,1.0,1.0,1.0,5.688879e-12
40402,"Itzbeen Pocket Nanny Baby Care Timer, Blue","OK, I know this seems like an idiot gift, but ...",4.0,OK I know this seems like an idiot gift but if...,1.0,1.0,1.0,9.222306e-12
58841,7A.M. ENFANT &quot;Blanket 212 Evolution&quot;...,Now that it served my son for almost four mont...,5.0,Now that it served my son for almost four mont...,1.0,1.0,1.0,1.121439e-11


In [106]:
correct_prediction_sentiment_data_train = len(data_train[data_train['sentiment'] == data_train['sentiment_prediction']])
correct_prediction_sentiment_data_test = len(data_test[data_test['sentiment'] == data_test['sentiments_prediction']])
print correct_prediction_sentiment_data_train
print correct_prediction_sentiment_data_test

116502
28225


In [109]:
train_acc_sentiment = correct_prediction_sentiment_data_train/float(len(data_train))
test_acc_sentiment = correct_prediction_sentiment_data_test/float(len(data_test))
print 'train accuracy sentiments',train_acc_sentiment
print 'test accuracy sentiments',test_acc_sentiment

train accuracy sentiments 0.962738924561
test accuracy sentiments 0.934324208017
